In [1]:
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('/Users/omarabuzeid/group1-project2/2021.csv', encoding ='latin1')

pd.set_option('display.max_colwidth', 1000)

df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1373819373090050048,1373669212271566858,1.616379e+12,2021-03-22 02:10:37,0,NaN,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'bluemoondance74', 'name': 'Reagan', 'id': '844015154643648512'}, {'screen_name': 'NASASpaceflight', 'name': 'Chris B - NSF', 'id': '21292523'}]",NaN,NaN,NaN,NaN
1,1,1373735946244431873,1373669212271566858,1.616359e+12,2021-03-21 20:39:07,0,NaN,@NASASpaceflight Hopefully will happen this decade,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'NASASpaceflight', 'name': 'Chris B - NSF', 'id': '21292523'}]",NaN,NaN,NaN,NaN
2,2,1373555480870621188,1373328330041229312,1.616316e+12,2021-03-21 08:42:00,0,NaN,@newscientist True,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'newscientist', 'name': 'New Scientist', 'id': '19658826'}]",NaN,NaN,NaN,NaN
3,3,1373507545315172357,1373263440391864323,1.616305e+12,2021-03-21 05:31:31,0,NaN,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'cleantechnica', 'name': 'CleanTechnica', 'id': '17060015'}]",NaN,NaN,NaN,NaN
4,4,1373492611231535111,1373357995288051718,1.616301e+12,2021-03-21 04:32:11,0,NaN,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'CathieDWood', 'name': 'Cathie Wood', 'id': '2361631088'}]",NaN,NaN,NaN,NaN


In [3]:
df.tweet = df.tweet.astype(str)


tweet_lens = []
for tweet in df.tweet:
    tweet_lens.append(len(tweet))
    
df_tweet_len = pd.DataFrame(tweet_lens)


df_tweet_len.columns = ['tweet_length']

df_tweet_len.head()

,tweet_length
0,86
1,50
2,18
3,128
4,170


In [4]:
df1 = pd.concat([df, df_tweet_len], axis=1)

df1.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tweet_length
0,0,1373819373090050048,1373669212271566858,1.616379e+12,2021-03-22 02:10:37,0,NaN,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,en,[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'bluemoondance74', 'name': 'Reagan', 'id': '844015154643648512'}, {'screen_name': 'NASASpaceflight', 'name': 'Chris B - NSF', 'id': '21292523'}]",NaN,NaN,NaN,NaN,86
1,1,1373735946244431873,1373669212271566858,1.616359e+12,2021-03-21 20:39:07,0,NaN,@NASASpaceflight Hopefully will happen this decade,en,[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'NASASpaceflight', 'name': 'Chris B - NSF', 'id': '21292523'}]",NaN,NaN,NaN,NaN,50
2,2,1373555480870621188,1373328330041229312,1.616316e+12,2021-03-21 08:42:00,0,NaN,@newscientist True,en,[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'newscientist', 'name': 'New Scientist', 'id': '19658826'}]",NaN,NaN,NaN,NaN,18
3,3,1373507545315172357,1373263440391864323,1.616305e+12,2021-03-21 05:31:31,0,NaN,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,en,[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'cleantechnica', 'name': 'CleanTechnica', 'id': '17060015'}]",NaN,NaN,NaN,NaN,128
4,4,1373492611231535111,1373357995288051718,1.616301e+12,2021-03-21 04:32:11,0,NaN,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",en,[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'CathieDWood', 'name': 'Cathie Wood', 'id': '2361631088'}]",NaN,NaN,NaN,NaN,170


In [5]:
df = df1
df.drop(columns=['id'], inplace=True)
df.head()

,Unnamed: 0,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tweet_length
0,0,1373669212271566858,1.616379e+12,2021-03-22 02:10:37,0,NaN,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'bluemoondance74', 'name': 'Reagan', 'id': '844015154643648512'}, {'screen_name': 'NASASpaceflight', 'name': 'Chris B - NSF', 'id': '21292523'}]",NaN,NaN,NaN,NaN,86
1,1,1373669212271566858,1.616359e+12,2021-03-21 20:39:07,0,NaN,@NASASpaceflight Hopefully will happen this decade,en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'NASASpaceflight', 'name': 'Chris B - NSF', 'id': '21292523'}]",NaN,NaN,NaN,NaN,50
2,2,1373328330041229312,1.616316e+12,2021-03-21 08:42:00,0,NaN,@newscientist True,en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'newscientist', 'name': 'New Scientist', 'id': '19658826'}]",NaN,NaN,NaN,NaN,18
3,3,1373263440391864323,1.616305e+12,2021-03-21 05:31:31,0,NaN,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'cleantechnica', 'name': 'CleanTechnica', 'id': '17060015'}]",NaN,NaN,NaN,NaN,128
4,4,1373357995288051718,1.616301e+12,2021-03-21 04:32:11,0,NaN,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'CathieDWood', 'name': 'Cathie Wood', 'id': '2361631088'}]",NaN,NaN,NaN,NaN,170


In [6]:
df = df[['date', 'tweet', 'tweet_length']]

df.head()

,date,tweet,tweet_length
0,2021-03-22 02:10:37,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,86
1,2021-03-21 20:39:07,@NASASpaceflight Hopefully will happen this decade,50
2,2021-03-21 08:42:00,@newscientist True,18
3,2021-03-21 05:31:31,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,128
4,2021-03-21 04:32:11,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",170


In [7]:
df.date.dtypes


df['date'] = pd.to_datetime(df['date'])

df.head()

<ipython-input-7-69f977feb35c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


,date,tweet,tweet_length
0,2021-03-22 02:10:37,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,86
1,2021-03-21 20:39:07,@NASASpaceflight Hopefully will happen this decade,50
2,2021-03-21 08:42:00,@newscientist True,18
3,2021-03-21 05:31:31,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,128
4,2021-03-21 04:32:11,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",170


In [8]:
df.set_index('date', inplace = True)

df.head()

,tweet,tweet_length
date,,
2021-03-22 02:10:37,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,86
2021-03-21 20:39:07,@NASASpaceflight Hopefully will happen this decade,50
2021-03-21 08:42:00,@newscientist True,18
2021-03-21 05:31:31,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,128
2021-03-21 04:32:11,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",170


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12339 entries, 2021-03-22 02:10:37 to 2010-06-04 18:31:57
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tweet         12339 non-null  object
 1   tweet_length  12339 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 289.2+ KB


In [10]:
xxx = df.tweet[0]
xxx

'@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!'

In [11]:
tweet_list = df.tweet.tolist()
tweet_list[0]

'@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!'

In [12]:
def clean_tweet_http(dirty_string):
    sep = "https://" 
    if sep in dirty_string:
        clean = dirty_string.split(sep, 1)[0]
    else:
        clean = dirty_string
    return clean

In [13]:
clean_tweets1 = []

for i in range(0, len(tweet_list)):
    x = clean_tweet_http(tweet_list[i])
    clean_tweets1.append(x)

In [14]:
clean_tweets1[:10]

['@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!',
 '@NASASpaceflight Hopefully will happen this decade',
 '@newscientist True',
 '@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars',
 '@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.',
 '@Swaggerheim @SciGuySpace Not trying to roast, just being as literal as possible',
 '@johnkrausphotos ð\x9f¤£ð\x9f¤£',
 '@AtomicsAstro @PPathole Yeah, but Iâ\x80\x99m English',
 '@SciGuySpace Not quite how I would tell the story, but itâ\x80\x99s probably worth a read',
 '@PPathole This is important']

In [15]:
# def clean_tweet_usernames(dirty_string):
#     sep = "https://" 
#     if sep in dirty_string:
#         clean = dirty_string.split(sep, 1)[0]
#     else:
#         clean = dirty_string
#     return clean

In [16]:
clean_tweets2 = []

# for i in range(0, len(clean_tweets1)):
for tweet in clean_tweets1:
    result = re.sub('@([A-Za-z0-9_]+) ', '', tweet)
    clean_tweets2.append(result)

In [17]:
clean_tweets2[:10]

['Going well. Lot of work for an extra 20 secs of Isp!',
 'Hopefully will happen this decade',
 'True',
 'I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars',
 'When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.',
 'Not trying to roast, just being as literal as possible',
 'ð\x9f¤£ð\x9f¤£',
 'Yeah, but Iâ\x80\x99m English',
 'Not quite how I would tell the story, but itâ\x80\x99s probably worth a read',
 'This is important']

In [18]:
clean_tweets3 = []

# for i in range(0, len(clean_tweets1)):
for tweet in clean_tweets2:
    result = re.sub('RT @([A-Za-z0-9_]+\:) ', '', tweet)
    clean_tweets3.append(result)
    
clean_tweets3[:10]

['Going well. Lot of work for an extra 20 secs of Isp!',
 'Hopefully will happen this decade',
 'True',
 'I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars',
 'When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.',
 'Not trying to roast, just being as literal as possible',
 'ð\x9f¤£ð\x9f¤£',
 'Yeah, but Iâ\x80\x99m English',
 'Not quite how I would tell the story, but itâ\x80\x99s probably worth a read',
 'This is important']

In [19]:
clean_tweets4 = []

for tweet in clean_tweets3:
    result = re.sub('\n', ' ', tweet)
    clean_tweets4.append(result)

    
clean_tweets4[:10]

['Going well. Lot of work for an extra 20 secs of Isp!',
 'Hopefully will happen this decade',
 'True',
 'I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars',
 'When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.',
 'Not trying to roast, just being as literal as possible',
 'ð\x9f¤£ð\x9f¤£',
 'Yeah, but Iâ\x80\x99m English',
 'Not quite how I would tell the story, but itâ\x80\x99s probably worth a read',
 'This is important']

In [20]:
clean_tweets5 = []

for tweet in clean_tweets4:
    result = re.sub(' https:?', '', tweet)
    clean_tweets5.append(result)
    
clean_tweets5[:10]

['Going well. Lot of work for an extra 20 secs of Isp!',
 'Hopefully will happen this decade',
 'True',
 'I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars',
 'When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.',
 'Not trying to roast, just being as literal as possible',
 'ð\x9f¤£ð\x9f¤£',
 'Yeah, but Iâ\x80\x99m English',
 'Not quite how I would tell the story, but itâ\x80\x99s probably worth a read',
 'This is important']

In [21]:
clean_tweets6 = []

for tweet in clean_tweets5:
    result = re.sub('@([A-Za-z0-9_]+)', '', tweet)
    clean_tweets6.append(result)

In [22]:

clean_tweets7 = []

for tweet in clean_tweets6:
    result = re.sub('\s([#][\w_-]+)', '', tweet)
    clean_tweets7.append(result)

In [23]:

clean_tweets8 = []

for tweet in clean_tweets7:
    result = re.sub('([#][\w_-]+)', '', tweet)
    clean_tweets8.append(result)

In [24]:
clean_tweets8[:10]

['Going well. Lot of work for an extra 20 secs of Isp!',
 'Hopefully will happen this decade',
 'True',
 'I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars',
 'When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.',
 'Not trying to roast, just being as literal as possible',
 'ð\x9f¤£ð\x9f¤£',
 'Yeah, but Iâ\x80\x99m English',
 'Not quite how I would tell the story, but itâ\x80\x99s probably worth a read',
 'This is important']

In [25]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [26]:
result

'Please ignore prior tweets, as that was someone pretending to be me :)  This is actually me.'

In [27]:
x1 = clean_tweet(result.lower())

In [28]:
def get_tweet_sentiment(tweet):
    
    analysis = TextBlob(clean_tweet(tweet.lower())) 
        # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive', analysis.sentiment.polarity
    elif analysis.sentiment.polarity == 0: 
        return 'neutral', analysis.sentiment.polarity
    else: 
        return 'negative', analysis.sentiment.polarity
    
get_tweet_sentiment(result)

('neutral', 0.0)

In [29]:
listy = []
for t in clean_tweets8:
    listy.append(get_tweet_sentiment(t))
    
len(listy)

12339

In [30]:
sentiments_list = []
polarities_list = []


for tweet in listy:
    sentiments_list.append(tweet[0])
    polarities_list.append(tweet[1])

In [31]:
sentiments = pd.DataFrame(sentiments_list)
polarities = pd.DataFrame(polarities_list)

len(df)

12339

In [32]:
len(sentiments)

12339

In [33]:
df2 = df

In [34]:
sentiments.index = df.index

In [35]:
polarities.index = df.index

In [36]:
df = pd.concat([df2, sentiments, polarities], axis=1)

In [37]:
df.head()

,tweet,tweet_length,0,0
date,,,,
2021-03-22 02:10:37,@bluemoondance74 @NASASpaceflight Going well. Lot of work for an extra 20 secs of Isp!,86,neutral,0.000000
2021-03-21 20:39:07,@NASASpaceflight Hopefully will happen this decade,50,neutral,0.000000
2021-03-21 08:42:00,@newscientist True,18,positive,0.350000
2021-03-21 05:31:31,@cleantechnica I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,128,positive,0.400000
2021-03-21 04:32:11,"@CathieDWood When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",170,positive,0.286111


In [38]:
df.columns = ['tweet', 'tweet_length', 'sentiment', 'polarity']

In [39]:
ct_series = pd.Series(clean_tweets8)
ct_series.index = df.index

ct_series[0]

'Going well. Lot of work for an extra 20 secs of Isp!'

In [40]:
ct_series.index

DatetimeIndex(['2021-03-22 02:10:37', '2021-03-21 20:39:07',
               '2021-03-21 08:42:00', '2021-03-21 05:31:31',
               '2021-03-21 04:32:11', '2021-03-21 04:25:23',
               '2021-03-21 04:23:40', '2021-03-21 04:22:32',
               '2021-03-21 04:21:32', '2021-03-21 04:19:13',
               ...
               '2011-12-21 10:48:29', '2011-12-21 10:42:52',
               '2011-12-21 10:30:05', '2011-12-21 10:26:51',
               '2011-12-04 03:33:52', '2011-12-03 08:22:07',
               '2011-12-03 08:20:28', '2011-12-01 10:29:04',
               '2011-12-01 09:55:11', '2010-06-04 18:31:57'],
              dtype='datetime64[ns]', name='date', length=12339, freq=None)

In [41]:
elon_df = pd.concat([df, ct_series], axis=1)
elon_df = elon_df.drop(columns=['tweet'])
elon_df.columns = ['tweet_length', 'sentiment', 'polarity', 'tweet']
elon_df.head()

,tweet_length,sentiment,polarity,tweet
date,,,,
2021-03-22 02:10:37,86,neutral,0.000000,Going well. Lot of work for an extra 20 secs of Isp!
2021-03-21 20:39:07,50,neutral,0.000000,Hopefully will happen this decade
2021-03-21 08:42:00,18,positive,0.350000,True
2021-03-21 05:31:31,128,positive,0.400000,I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars
2021-03-21 04:32:11,170,positive,0.286111,"When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great."


In [42]:
le = LabelEncoder()
sen = elon_df.sentiment
sen = le.fit_transform(sen)
sen = pd.Series(sen-1)
sen.index = elon_df.index
elon_df = pd.concat([elon_df, sen], axis=1)
elon_df.columns = ['tweet_length','sentiment', 'polarity', 'tweet', 'encoded_sentiment',]


elon_df.head()

,tweet_length,sentiment,polarity,tweet,encoded_sentiment
date,,,,,
2021-03-22 02:10:37,86,neutral,0.000000,Going well. Lot of work for an extra 20 secs of Isp!,0
2021-03-21 20:39:07,50,neutral,0.000000,Hopefully will happen this decade,0
2021-03-21 08:42:00,18,positive,0.350000,True,1
2021-03-21 05:31:31,128,positive,0.400000,I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars,1
2021-03-21 04:32:11,170,positive,0.286111,"When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great.",1


In [43]:
elon_df = elon_df[['tweet_length', 'encoded_sentiment', 'sentiment', 'polarity', 'tweet']]
elon_df.describe()

,tweet_length,encoded_sentiment,polarity
count,12339.000000,12339.000000,12339.000000
mean,90.777454,0.328957,0.119987
std,66.422119,0.699777,0.258747
min,1.000000,-1.000000,-1.000000
25%,39.000000,0.000000,0.000000
50%,74.000000,0.000000,0.000000
75%,124.000000,1.000000,0.250000
max,430.000000,1.000000,1.000000


In [44]:
elon_df.head()

,tweet_length,encoded_sentiment,sentiment,polarity,tweet
date,,,,,
2021-03-22 02:10:37,86,0,neutral,0.000000,Going well. Lot of work for an extra 20 secs of Isp!
2021-03-21 20:39:07,50,0,neutral,0.000000,Hopefully will happen this decade
2021-03-21 08:42:00,18,1,positive,0.350000,True
2021-03-21 05:31:31,128,1,positive,0.400000,I am accumulating resources to help make life multiplanetary &amp; extend the light of consciousness to the stars
2021-03-21 04:32:11,170,1,positive,0.286111,"When vast amounts of manufacturing are needed, as in robotaxis, this slows down rate of introduction, so maybe more like winner-takes-a-quarter. Still great."
